### Reading the data

In [74]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import csv
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import warnings
warnings.filterwarnings("ignore")


In [75]:
### Reading the Train event data
df_events = pd.read_csv("train_event_data.csv", dtype={'device_id': np.str, 'latitude': np.float, 'longitude':np.float,'event_id': np.str})
### Reading the APP Event Data 
df_app_event = pd.read_csv('app_events.csv',dtype={'event_id': np.str, 'app_id': np.str, 'is_active':np.int64,'is_installed':np.int64})
df_mobile_brand = pd.read_csv('train_mobile_brand.csv',dtype={'device_id': np.str})
df_app_metadata = pd.read_csv('app_events_meta_data.csv',encoding='utf-8',quoting=csv.QUOTE_NONE)


In [55]:

df_events['datetimestamp'] = pd.to_datetime(df_events['datetimestamp'])
df_events['event_day'] = df_events['datetimestamp'].dt.day 

In [56]:
df_events.head()

,device_id,gender,age,group_train,event_id,datetimestamp,latitude,longitude,event_day
0,-7548291590301750000,M,33,M32+,2369465,2016-05-03 15:55:35,33.98,116.79,3.0
1,-7548291590301750000,M,33,M32+,1080869,2016-05-03 06:07:16,33.98,116.79,3.0
2,-7548291590301750000,M,33,M32+,1079338,2016-05-04 03:28:02,33.98,116.79,4.0
3,-7548291590301750000,M,33,M32+,1078881,2016-05-04 02:53:08,33.98,116.79,4.0
4,-7548291590301750000,M,33,M32+,1068711,2016-05-03 15:59:35,33.98,116.79,3.0


In [57]:
 = df_events.loc[~df_events.isna()['event_id']]

### without Event data
df_train_event_without = df_events.loc[df_events.isna()['event_id']]

### Some device IDs have multiple events in a day or over a period of days.

### Find the average number of events – find the percentage of time the mobile phone was active by calculating the number of events for a device ID

In [47]:
df_train_event_with['event_avg'] = df_train_event_with.groupby(['device_id', 'event_day'])['event_id'].transform('count')/df_train_event_with.groupby(['device_id'])['event_id'].transform('count')


### For a person going from home to the office in the morning and returning at night, figure out the difference in the latitudes and longitudes over a morning period versus the evening and the night periods

In [58]:
df_train_event_with['latitude_difference'] = df_train_event_with.groupby(['device_id', 'event_day'])['latitude'].transform(lambda x: x.max() - x.min())
df_train_event_with['longitude_difference'] = df_train_event_with.groupby(['device_id', 'event_day'])['longitude'].transform(lambda x: x.max() - x.min())


C:\Users\SESA662668\AppData\Local\Temp\ipykernel_18612\2630924574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_event_with['latitude_difference'] = df_train_event_with.groupby(['device_id', 'event_day'])['latitude'].transform(lambda x: x.max() - x.min())
C:\Users\SESA662668\AppData\Local\Temp\ipykernel_18612\2630924574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_event_with['longitude_difference'] = df_train_event_with.groupby(['device_id', 'event_day'])['longitude'].transform

In [59]:
df_train_event_with.head()

,device_id,gender,age,group_train,event_id,datetimestamp,latitude,longitude,event_day,latitude_difference,longitude_difference
0,-7548291590301750000,M,33,M32+,2369465,2016-05-03 15:55:35,33.98,116.79,3.0,0.0,0.0
1,-7548291590301750000,M,33,M32+,1080869,2016-05-03 06:07:16,33.98,116.79,3.0,0.0,0.0
2,-7548291590301750000,M,33,M32+,1079338,2016-05-04 03:28:02,33.98,116.79,4.0,0.0,0.0
3,-7548291590301750000,M,33,M32+,1078881,2016-05-04 02:53:08,33.98,116.79,4.0,0.0,0.0
4,-7548291590301750000,M,33,M32+,1068711,2016-05-03 15:59:35,33.98,116.79,3.0,0.0,0.0


### You can also think of creating features such as a median latitude and a median longitude over a period of different events, which is given as per the device ID.

In [60]:
df_train_event_with['latitude_medium'] = df_train_event_with.groupby(['device_id'])['latitude'].transform('median')
df_train_event_with['longitude_medium'] = df_train_event_with.groupby(['device_id'])['longitude'].transform('median')


C:\Users\SESA662668\AppData\Local\Temp\ipykernel_18612\4087469830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_event_with['latitude_medium'] = df_train_event_with.groupby(['device_id'])['latitude'].transform('median')
C:\Users\SESA662668\AppData\Local\Temp\ipykernel_18612\4087469830.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_event_with['longitude_medium'] = df_train_event_with.groupby(['device_id'])['longitude'].transform('median')


In [61]:
df_train_event_with.head()

,device_id,gender,age,group_train,event_id,datetimestamp,latitude,longitude,event_day,latitude_difference,longitude_difference,latitude_medium,longitude_medium
0,-7548291590301750000,M,33,M32+,2369465,2016-05-03 15:55:35,33.98,116.79,3.0,0.0,0.0,33.98,116.79
1,-7548291590301750000,M,33,M32+,1080869,2016-05-03 06:07:16,33.98,116.79,3.0,0.0,0.0,33.98,116.79
2,-7548291590301750000,M,33,M32+,1079338,2016-05-04 03:28:02,33.98,116.79,4.0,0.0,0.0,33.98,116.79
3,-7548291590301750000,M,33,M32+,1078881,2016-05-04 02:53:08,33.98,116.79,4.0,0.0,0.0,33.98,116.79
4,-7548291590301750000,M,33,M32+,1068711,2016-05-03 15:59:35,33.98,116.79,3.0,0.0,0.0,33.98,116.79


In [76]:
df_app_metadata.head()

,app_id,label_id,category
0,app_id,label_id,category
1,7324884708820027918,251,Finance
2,-4494216993218550286,251,Finance
3,6058196446775239644,406,unknown
4,6058196446775239644,407,DS_P2P net loan


### You can group the categories of the apps. This should play a vital role. For example, young females would use more apps in the categories of fashion and photography.


In [77]:

# Mapping dictionary or DataFrame
mapping = {
    'Industry tag': 'Industry',
    'Property Industry 1.0': 'Property',
    'Nature 1': 'Nature',
    'Personal Effectiveness 1': 'Personal Effectiveness',
    'Property Industry 2.0': 'Property',
    '1 free': 'Free',
    'Relatives 1': 'Relatives',
    'Science and Technology 1': 'Science and Technology',
    'Services 1': 'Services',
    'unknown': 'Unknown',
    'Customized 1': 'Customized',
    'Irritation / Fun 1': 'Fun',
    'The elimination of class': 'Education',
    'game': 'Gaming',
    'Passion 1': 'Passion',
    'Flight Shooting': 'Gaming',
    'Cozy 1': 'Comfort',
    'Custom label': 'Custom',
    'And the Church': 'Religion',
    'Internet banking': 'Financial Services',
    'Third party payment': 'Financial Services',
    'High Flow': 'Financial Services',
    'Moderate profitability': 'Financial Services',
    'Low risk': 'Financial Services',
    'Financial Services': 'Financial Services',
    'Low liquidity': 'Financial Services',
    'Low income': 'Financial Services',
    'Low Risk': 'Financial Services',
    'Science and Technology': 'Science and Technology',
    'pursue': 'Personal Development',
    'mobile bank': 'Financial Services',
    'Bank financing': 'Financial Services',
    'fixed income': 'Financial Services',
    'Liquid medium': 'Financial Services',
    'Low profitability': 'Financial Services',
    'Direct Bank': 'Financial Services',
    'Direct Banking': 'Financial Services',
    'Smart Shopping 1': 'Shopping',
    'Finance': 'Financial Services',
    'Wealth Management': 'Financial Services',
    'Debit and credit': 'Financial Services',
    'Insurance': 'Financial Services',
    'Lower liquidity': 'Financial Services',
    'Medium risk': 'Financial Services',
    'IMF': 'Financial Services',
    'P2P': 'Financial Services',
    'Higher income': 'Financial Services',
    'High risk': 'Financial Services',
    'Securities': 'Financial Services',
    'DS_P2P net loan': 'Financial Services',
    'Trendy / cool 1': 'Trendy/Cool',
    'fund': 'Financial Services',
    'Monetary Fund': 'Financial Services',
    'High mobility': 'Travel',
    'Pay': 'Payment',
    'Quality 1': 'Quality',
    '1 reputation': 'Reputation',
    'Pursuit 1': 'Personal Development',
    'Other Read': 'Reading',
    'Total Cost 1': 'Finance',
    'Financial Supermarket': 'Financial Services',
    'Chess categories': 'Gaming',
    'Buy class': 'Shopping',
    'read': 'Reading',
    'literature': 'Reading',
    'Customization': 'Custom',
    'service': 'Services',
    'Personal Effectiveness': 'Personal Effectiveness',
    'Smart Shopping': 'Shopping',
    'Trendy / cool': 'Trendy/Cool',
    'Relatives': 'Relatives',
    'free': 'Free',
    'comfortable': 'Comfort',
    'Online booking class': 'Travel',
    'stock': 'Financial Services',
    'A shares': 'Financial Services',
    'High profitability': 'Financial Services',
    'Casual puzzle categories': 'Gaming',
    '1 vitality': 'Health',
    'Condition of the vehicles': 'Automotive',
    'Consumer loans': 'Financial Services',
    'Direct Insurance': 'Insurance',
    'Racing (RAC)': 'Gaming',
    'Equity Fund': 'Financial Services',
    'Lottery ticket': 'Gaming',
    'Pirated content': 'Other',
    'Stock Futures': 'Financial Services',
    'Commodity Futures': 'Financial Services',
    'Life Insurance': 'Insurance',
    'music': 'Music',
    'Listening to books category': 'Books',
    'other': 'Other',
    'Financial Information': 'Financial Services',
    'exchange rate': 'Finance',
    'Occupational identity': 'Career',
    'College Students': 'Education',
    'Hardware Related': 'Technology',
    'Cards RPG': 'Gaming',
    'Crowdfunding financing': 'Finance',
    'Simple 1': 'Simple',
    'the film': 'Movies',
    'SLG (strategy)': 'Gaming',
    'ARPG': 'Gaming',
    'round': 'Gaming',
    'Noble 1': 'Luxury',
    'User Community': 'Social',
    'Parkour avoid class': 'Sports',
    'Precious Metals': 'Finance',
    'futures': 'Financial Services',
    'Shootout Shooting (STG)': 'Gaming',
    'Animation aggregate class': 'Movies',
    'Classical 1': 'Classical',
    'Business simulation': 'Business',
    'Ping': 'Communication',
    'safety Insurance': 'Insurance',
    'Vermicelli': 'Food',
    "Read the operator's": 'Reading',
    'H shares': 'Financial Services',
    'Financial Futures': 'Financial Services',
    'Crowdfunding': 'Finance',
    'MMO': 'Gaming',
    'Exchange': 'Financial Services',
    'vitality': 'Health',
    'Tower Defense': 'Gaming',
    'Other shares': 'Financial Services',
    'Peace - Search': 'Search',
    'Sports': 'Sports',
    'Antique collection': 'Collectibles',
    'Class animation community': 'Movies',
    'MOBA': 'Gaming',
    'Simple': 'Simple',
    'Trust': 'Other'
}



# Map subcategories to categories
df_app_metadata['category'] = df_app_metadata['category'].map(mapping).fillna('Other')
    
# Print the result
df_app_metadata.category.unique()


array(['Other', 'Financial Services', 'Unknown', 'Gaming', 'Finance',
       'Personal Development', 'Science and Technology', 'Custom',
       'Services', 'Shopping', 'Personal Effectiveness', 'Free', 'Health',
       'Trendy/Cool', 'Relatives', 'Comfort', 'Simple', 'Communication',
       'Insurance', 'Search', 'Property', 'Industry', 'Payment',
       'Customized', 'Luxury', 'Passion', 'Fun', 'Quality', 'Reputation',
       'Nature', 'Classical', 'Career', 'Education', 'Automotive',
       'Collectibles', 'Travel', 'Religion', 'Reading', 'Sports',
       'Business', 'Music', 'Books', 'Social', 'Food', 'Movies',
       'Technology'], dtype=object)

In [79]:
df_app_metadata.to_csv('df_app_metadata.csv')


In [81]:
df_app_metadata.category.nunique()

46

### DBSCAN to find the cluster for Geolocation

In [63]:
df = df_train_event_with.groupby(['latitude','longitude']).size().reset_index().rename(columns={0:'count'})

In [67]:
X = df[['latitude', 'longitude']]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [68]:
kms_per_radian = 6371.0088
epsilon = 10/kms_per_radian

dbscan = DBSCAN(eps= epsilon, min_samples= 10, algorithm = 'ball_tree', metric = 'haversine')

# Perform clustering
clusters = dbscan.fit_predict(X_scaled)

In [69]:
df['cluster_label'] = clusters

# Print the number of clusters
num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
print(f'Number of clusters: {num_clusters}')

# Print the number of noise points
num_noise_points = list(clusters).count(-1)
print(f'Number of noise points: {num_noise_points}')

# Print the samples in each cluster
cluster_counts = df['cluster_label'].value_counts()
print('Samples in each cluster:')
print(cluster_counts)

Number of clusters: 85
Number of noise points: 27733
Samples in each cluster:
-1     27733
 63     1020
 9       882
 25      460
 77      354
       ...  
 66        8
 15        7
 41        6
 44        6
 46        5
Name: cluster_label, Length: 86, dtype: int64


In [70]:
df.drop(['count'], axis=1,inplace=True)

In [71]:
df_train_event_with = pd.merge(df_train_event_with, df, on=['latitude', 'longitude'], how='left')


In [72]:
df_train_event_with.head()

,device_id,gender,age,group_train,event_id,datetimestamp,latitude,longitude,event_day,latitude_difference,longitude_difference,latitude_medium,longitude_medium,cluster_label
0,-7548291590301750000,M,33,M32+,2369465,2016-05-03 15:55:35,33.98,116.79,3.0,0.0,0.0,33.98,116.79,-1
1,-7548291590301750000,M,33,M32+,1080869,2016-05-03 06:07:16,33.98,116.79,3.0,0.0,0.0,33.98,116.79,-1
2,-7548291590301750000,M,33,M32+,1079338,2016-05-04 03:28:02,33.98,116.79,4.0,0.0,0.0,33.98,116.79,-1
3,-7548291590301750000,M,33,M32+,1078881,2016-05-04 02:53:08,33.98,116.79,4.0,0.0,0.0,33.98,116.79,-1
4,-7548291590301750000,M,33,M32+,1068711,2016-05-03 15:59:35,33.98,116.79,3.0,0.0,0.0,33.98,116.79,-1


### Saving the file after completation of the feature engineering

In [73]:


df_train_event_with.to_csv('train_with_event.csv')
df_train_event_without.to_csv('train_without_event.csv')